In [7]:
import os
import mido
import utils ##
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

import torch
from torch.utils.data import DataLoader

## Importar Datos

In [13]:
df_audios = pd.read_csv('datos_procesados/valid_samples.csv', index_col=0)
audio_names = list(df_audios.index)

## Dataset

- Definición del objeto Dataset para importar los datos

In [46]:
class TarareoMIDIDataset(torch.utils.data.Dataset):
    def __init__(self, tarareo_directory: str, midi_directory: str, samples: list, extensions: tuple[str, str]):
        self._tarareo_directory = tarareo_directory
        self._midi_directory = midi_directory
        
        self._samples = samples #  Nombre del elemento de grabación
        self._extensions = extensions #  extension del nombre del tarareo y vector midi. Ejm audio_name[extension] (incluye formato)

        # Si usamos PADDING, será conveniente tener un vocabulario como si se tratara de una traducción en NLP
        # Esto ayuda a ordenar las etiquetas por orden de frecuencia
        # Colocando en las primeras casillas a las etiquetas de inicio y final
        # self._vocabulary = vocabulary


    def __len__(self):
        return len(self._samples)

    def __getitem__(self, idx):
        # Recuperar nombre de la muestra
        file_name = self._samples[idx]
        tarareo_name = file_name + self._extensions[0]
        midi_name = file_name + self._extensions[1]

        # Importar
        tarareo_path = os.path.join(self._tarareo_directory, tarareo_name)
        tarareo = np.load(tarareo_path)

        midi_path = os.path.join(self._midi_directory, midi_name)
        midi = np.load(midi_path)

        # Agregar <SOS> y <EOS> ?
        return torch.tensor(tarareo, dtype=torch.float32), torch.tensor(midi, dtype=torch.int)

- Definición de `collate_fn` para crear batches y homologar las dimensiones de audios de distinta duración.

In [45]:
def audio_vector_collate_fn(batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
    """Función para agregar padding a los audios y vectores de etiquetas por lote.

    :param batch: Una lista de tuplas donde cada tupla contiene un audio y su vector de etiquetas.
    :return: Una tupla que contiene dos tensores:
             1) Un tensor que contiene todos los audios del lote, apilados juntos. Forma: [batch_size, L_max, N_max].
             2) Un tensor que contiene todos los vectores de etiquetas del lote, rellenados con zeros para que tengan la misma longitud. 
                Forma: [batch_size, K_max].
    """
    audios, labels = zip(*batch)
    
    # Determinar las dimensiones máximas
    L_max = max(audio.size(0) for audio in audios)
    N_max = max(audio.size(1) for audio in audios)
    K_max = max(label.size(0) for label in labels)
    
    # Inicializar los tensores con padding
    padded_audios = torch.zeros(len(audios), L_max, N_max, dtype=torch.float32)
    padded_labels = torch.zeros(len(labels), K_max, dtype=torch.int64)
    
    for i, (audio, label) in enumerate(batch):
        L = audio.size(0)
        N = audio.size(1)
        K = label.size(0)
        
        # Copiar el audio y el vector de etiquetas a los tensores con padding
        padded_audios[i, :L, :N] = audio
        padded_labels[i, :K] = label
    
    return padded_audios, padded_labels

## Separación de conjuntos

In [72]:
tarareo_directory = "datos_procesados/tarareos/ventanas/"
midi_directory = 'datos_procesados/midis/target_vectors/'
extensions = ('_frames.npy', '_target.npy')
dataset = TarareoMIDIDataset(tarareo_directory=tarareo_directory, midi_directory=midi_directory, samples=audio_names, extensions=extensions)

In [73]:
props = [0.7, 0.2, 0.1]
train_size = int(props[0] * len(dataset))
val_size = int(props[1] * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, props) # [train_size, val_size, test_size]

In [74]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=audio_vector_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=audio_vector_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=audio_vector_collate_fn)

## NEXT

In [172]:
def convout_calc(input_dim, kernel, stride, dilation=1):
    new_dim = np.floor((input_dim - 1 - dilation*(kernel-1)) / stride + 1)
    return int(new_dim)

In [ ]:
class ConvSeq2Seq(torch.nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim): #conv_out_channels, conv_kernel, pool_kernel,
        super(ConvSeq2Seq, self).__init__()
        self._input_dim = input_dim
        self._output_dim = output_dim
        self._hidden_dim = hidden_dim

        ##### Capa Conv1D
        self.conv2d = torch.nn.Conv2d(in_channels=1, 
                                out_channels=1, 
                                kernel_size=(int(1378//16),4), # L=1378 muestras/ventana (//16 = 86) | conv_kernel
                                stride=(1, 1))
        # Calcular dimensiones de salida si se quiere usar conv_kernel como un parámetro        
        self.conv2_H_dim = convout_calc(input_dim, kernel=int(1378//16), stride=1)

        ##### Capa MaxPooling
        self.MaxPool2d = torch.nnnn.MaxPool2d(kernel_size=(int(86//4),2)) # (86//4 = 21) | pool_kernel
        # Calcular dimensiones de salida si se quiere usar pool_kernel como un parámetro 
        self.pool_H_dim = convout_calc(self.conv2_H_dim, kernel=int(86//4), stride=int(86//4))
        
        ###### Capa de Flatten
        # self.flatten = nn.Flatten()

        ##### Encoder LSTM
        self.encoder_lstm = torch.nn.LSTM(input_size=self.pool_H_dim,  # Se ajusta para el tamaño de las ventanas y strides
                                    hidden_size=hidden_dim,
                                    batch_first=True)
        
        # Decoder LSTM
        self.decoder_lstm = torch.nn.LSTM(input_size=hidden_dim,
                                    hidden_size=hidden_dim,
                                    batch_first=True)

        # Capa fully connected
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x, target_len):
        # Conv1D
        conv_out = self.conv2d(x)

        # Conv1D
        pool_out = self.conv2d(conv_out)
        pool_out = pool_out.squeeze(1) # Elimina el canal muletilla de Conv
        pool_out_t = pool_out.transpose(1,2) # Transpone para ajustarse a las dim de LSTM input

        # # Flatten
        # flattened = self.flatten(conv_out)
        

        # Encoder LSTM
        encoder_outputs, (hidden, cell) = self.encoder_lstm(pool_out_t)

        # Decoder LSTM
        batch_size = x.size(0)
        decoder_input = torch.zeros((batch_size, 1, self._hidden_dim.size(2))) # Inicializar con ceros
        
        decoder_outputs = []
        for t in range(target_len):
            output, (hidden, cell) = self.decoder_lstm(decoder_input, (hidden, cell))
            output = self.fc(output)
            decoder_outputs.append(output)
            decoder_input = output
        
        return decoder_outputs #torch.stack(decoder_outputs, dim=1).squeeze(2)

# Ejemplo de uso
input_dim = 64  # Ejemplo, dimensión de entrada
output_dim = 3  # Predicción de etiquetas 0, 1, 2
conv_out_channels = 32
lstm_hidden_dim = 128
lstm_layers = 2
batch_size = 10
seq_len = 100  # Ejemplo de longitud de secuencia

model = ConvSeq2Seq(input_dim, output_dim, conv_out_channels, lstm_hidden_dim, lstm_layers)

# Datos de ejemplo
input_data = torch.randn(batch_size, input_dim, seq_len)
target_len = 30  # Ejemplo de longitud de secuencia de salida

output = model(input_data, target_len)
print(output.shape)  # Esperado: (batch_size, target_len, output_dim)


In [ ]:
# Función de entrenamiento
def train_seq2seq(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for src, trg in train_loader:
            src, trg = src.unsqueeze(1), trg  # Añadir dimensión de canal
            optimizer.zero_grad()
            output = model(src, trg)
            output = output[:, 1:].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        val_loss = evaluate_seq2seq(model, val_loader, criterion)
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Función de evaluación
def evaluate_seq2seq(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for src, trg in val_loader:
            src, trg = src.unsqueeze(1), trg  # Añadir dimensión de canal
            output = model(src, trg, 0)  # No usar teacher forcing durante la evaluación
            output = output[:, 1:].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
            val_loss += loss.item()
    return val_loss / len(val_loader)

# Configuración de optimizador y función de pérdida
criterion = torch.nn.CrossEntropyLoss(ignore_index=0)  # Ignorar el padding
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenar el modelo
train_seq2seq(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)
